# Description

## Objective: 
In-depth backtesting of a quantitative trading strategy over a 35-year period, which utilizes Fama-MacBeth cross-sectional regressions and Random Forest Model to train, validate, and test the trading model of 11 factors of four categories.

## Details:
### In-sample Training
1. The whole backtesting period is 35 years with 300 months (15 years) in-sample training starting from Jan 1983 to Dec 2019 and 120 months (10 years) out-of sample testing  starting from Jan 2020 to Dec 2019.

2. The universe of stocks is NYSE/Nasdaq stocks, only common stocks (not preferred, ETFs, etc.).

3. Each stock must have a price that exceeds $5 per share and a market capitalization of equity of at least $100 million at the beginning of every forecast year (January of each year), to avoid trading illiquid stocks.

4. Uniformize / standardiz and filter values of each of chosen factors using z-scores, relative to the industry average and standard deviation of each month.

5. Use Fama-MacBeth regressions to estimate the factor premia on each factor during each estimation window, then compute the average factor premia across all 
windows and the t-statistic of this average (i.e., the Fama-MacBeth average and the Fama-MacBeth t-statistic). These represent the average profitability and the consistency of 
profitability of each factor, respectively. 

6. Drop factors from the final model if they end up with a backtest t-statistic that is low (e.g., below 1) or the wrong sign (e.g., negative for the E/P ratio, which should positively predict returns). However, may choose to keep a signal with a low t-statistic or even a wrong sign if there seems to be a strong economic story or a compelling research article that indicates it should work well when tried over many years. A rough “rule-of-thumb” is to keep at least 5-10 factors.

### Out-of-sample Testing
1. Use Fama-MacBeth linear regression (predicted return = alpha + beta's * factor's) to score stocks / predict returns then rank the portfolios in 10 quintiles and construct hedge portfolios by longing the top 10% and shorting the bottom 10% portfolios at each month

2. Use Random Forest model score stocks / predict returns then rank the portfolios in 10 quintiles and construct hedge portfolios by longing the top 10% and shorting the bottom 10% portfolios at each month

3. Taking the average of both Fama-MacBeth linear regression and Random Forest model score stocks / predict returns then rank the portfolios in 10 quintiles and construct hedge portfolios by longing the top 10% and shorting the bottom 10% portfolios at each month

4. Analyze the performance of the three constructed hedge portfolios using CAPM model and Fama-French Four Factor model 

# Codes

In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf
from sklearn import linear_model
import statsmodels.api as sm

## In-sample Training

### Data Preprocessing

In [2]:
crsp = pd.read_sas("crsp_industry_2.sas7bdat", encoding="ISO-8859-1")
crsp['monthid'] = (crsp.DATE.dt.year-1985)*12 + crsp.DATE.dt.month
crsp['PRC'] = np.abs(crsp['PRC'])
crsp['MktCap'] = crsp['SHROUT']*crsp['PRC']/1000 # Note that the unit shares outstanding is in thousand

In [3]:
# Each stock has the data from Jan 1983 to Dec 2019 to train the model
validPeriod = pd.merge(crsp[crsp['DATE']=='1983-03-31'], crsp[crsp['DATE']=='2019-12-31'], on='PERMNO', how='inner')[['PERMNO']]
crsp = pd.merge(crsp, validPeriod, on = ['PERMNO'])

# Price > 5 and Market Cap >= $100 M at the start of each January in the out-of sample testing period (2010 - 2020)
validStock = crsp[(crsp['DATE'].dt.year>=2010) & (crsp['DATE'].dt.month==1) & (crsp['PRC']>5) & (crsp['MktCap']>=100)][['PERMNO']].drop_duplicates()
crsp = pd.merge(crsp, validStock, on=['PERMNO'])

# Add industry code/id
indID = crsp[crsp['DATE']=='2019-12-31'][['PERMNO', 'NAICS']]
indID['indID'] = indID['NAICS'].str.extract(r'(\d\d)')
crsp = pd.merge(crsp, indID[['PERMNO', 'indID']], on=['PERMNO'])

# Match PERMNO with GVKEY
permno_gvkey = pd.read_sas("permno_gvkey.sas7bdat", encoding="ISO-8859-1")
gvkeys = permno_gvkey['GVKEY'].copy().drop_duplicates()
# Extract GVKEY that is used to download finncial date from CRSP as the company codes
# gvkeys.to_csv('gvkey.txt', index=False, header=False)
crsp = pd.merge(crsp, permno_gvkey[['PERMNO', 'GVKEY']], on=['PERMNO'], how='inner')
crsp

,PERMNO,DATE,SHRCD,EXCHCD,NAICS,PRC,VOL,RET,SHROUT,monthid,MktCap,indID,GVKEY
0,10145.0,1983-01-31,11.0,1.0,NaN,34.625000,49328.0,0.069498,35250.0,-23,1220.531250,42,001300
1,10145.0,1983-02-28,11.0,1.0,NaN,37.500000,42603.0,0.100361,35250.0,-22,1321.875000,42,001300
2,10145.0,1983-03-31,11.0,1.0,NaN,44.500000,59780.0,0.186667,35250.0,-21,1568.625000,42,001300
3,10145.0,1983-04-29,11.0,1.0,NaN,45.500000,37666.0,0.022472,35250.0,-20,1603.875000,42,001300
4,10145.0,1983-05-31,11.0,1.0,NaN,48.875000,36379.0,0.087363,49521.0,-19,2420.338875,42,001300
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117535,83601.0,2019-09-30,11.0,1.0,212299,36.049999,60992.0,0.045821,54871.0,417,1978.099508,21,011600
117536,83601.0,2019-10-31,11.0,1.0,212299,36.810001,44614.0,0.021082,55924.0,418,2058.562517,21,011600
117537,83601.0,2019-11-29,11.0,1.0,212299,38.299999,28758.0,0.040478,55924.0,419,2141.889157,21,011600
117538,83601.0,2019-12-31,11.0,1.0,212299,42.180000,58234.0,0.107572,55094.0,420,2323.864937,21,011600


### Add factors

Here, we applied three-month lag rule to address the time lag, a delay between when financial data is reported and when it's reflected in stock prices, by matching financial data with stock prices with a three-month gap. This means that if a company reports its financials for a quarter ending on March 31 (referred to as "DATADATE"), the financial data for that quarter would be associated with the stock months of July, August, and September of the same year.

In practical terms, this means that the financial performance reported for the first quarter (ending in March) would be considered when analyzing stock price movements in the subsequent months of July, August, and September. Each month within this matching quarter (July to September in this example) would receive the same quarterly financial value for analysis.

In [4]:
factors = pd.read_sas("completeFac.sas7bdat", encoding="ISO-8859-1")
factors['monthid'] = (factors.DATADATE.dt.year-1985)*12 + factors.DATADATE.dt.month

In [5]:
# Apply three-month lag rule
comp_crsp = sqldf(
    "SELECT a.GVKEY, a.monthid, a.PRC, a.VOL, a.RET, a.SHROUT, a.indID, a.MktCap,\
    b.GVKEY AS GVKEY_2, b.monthid AS monthid_2,\
    b.ACTQ, b.AJEXQ, b.ATQ, b.CHEQ, b.COGSQ, b.CSHOQ, b.CSHPRQ, b.DLCQ,\
    b.DLTTQ, b.DVPQ, b.INVTQ, b.LCTQ, b.LTQ, b.NIQ, b.OIADPQ, b.OIBDPQ,\
    b.PSTKQ, b.REVTQ, b.SEQQ, b.TXTQ, b.CEQQ\
    FROM crsp as a\
    LEFT JOIN factors as b\
    ON a.GVKEY = b.GVKEY and a.monthid >= b.monthid+4 and a.monthid <= b.monthid+6"
)

comp_crsp = comp_crsp.dropna()
comp_crsp.sort_values(["GVKEY", "monthid"])
comp_crsp.reset_index(drop=True, inplace=True)
comp_crsp

,GVKEY,monthid,PRC,VOL,RET,SHROUT,indID,MktCap,GVKEY_2,monthid_2,...,LCTQ,LTQ,NIQ,OIADPQ,OIBDPQ,PSTKQ,REVTQ,SEQQ,TXTQ,CEQQ
0,001300,43,35.500000,61092.0,0.028986,149949.0,42,5323.189500,001300,39.0,...,3303.000,6936.000,112.000,220.000,326.000,0.0,2942.000,3191.000,43.000,3191.000
1,001300,44,33.000000,126959.0,-0.057746,149949.0,42,4948.317000,001300,39.0,...,3303.000,6936.000,112.000,220.000,326.000,0.0,2942.000,3191.000,43.000,3191.000
2,001300,45,33.750000,45305.0,0.022727,149949.0,42,5060.778750,001300,39.0,...,3303.000,6936.000,112.000,220.000,326.000,0.0,2942.000,3191.000,43.000,3191.000
3,001300,46,34.125000,54970.0,0.011111,149949.0,42,5117.009625,001300,42.0,...,3442.000,7052.000,128.000,239.000,349.000,0.0,3096.000,3245.000,40.000,3245.000
4,001300,47,34.125000,45397.0,0.013187,149949.0,42,5117.009625,001300,42.0,...,3442.000,7052.000,128.000,239.000,349.000,0.0,3096.000,3245.000,40.000,3245.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90463,011600,417,36.049999,60992.0,0.045821,54871.0,21,1978.099508,011600,413.0,...,698.020,1562.402,37.738,38.140,62.099,0.0,938.842,831.246,9.151,831.246
90464,011600,418,36.810001,44614.0,0.021082,55924.0,21,2058.562517,011600,413.0,...,698.020,1562.402,37.738,38.140,62.099,0.0,938.842,831.246,9.151,831.246
90465,011600,419,38.299999,28758.0,0.040478,55924.0,21,2141.889157,011600,413.0,...,698.020,1562.402,37.738,38.140,62.099,0.0,938.842,831.246,9.151,831.246
90466,011600,420,42.180000,58234.0,0.107572,55094.0,21,2323.864937,011600,416.0,...,482.674,1475.191,-4.776,13.668,37.845,0.0,838.659,787.973,-0.185,787.973


In [6]:
facDF = comp_crsp[['GVKEY', 'indID', 'monthid', 'RET']].copy()
facDF = facDF[facDF['monthid']>0]

# Tobin’s Q Ratio
facDF['ToQ'] = (comp_crsp['MktCap']+comp_crsp['PSTKQ']+comp_crsp['LCTQ']-comp_crsp['ACTQ']+comp_crsp['DLTTQ']) / comp_crsp['ATQ']

# Stochastic Oscillator
facDF['STO'] = ((comp_crsp['PRC']-comp_crsp['PRC'].rolling(window=14).min()) / (comp_crsp['PRC'].rolling(window=14).max()-comp_crsp['PRC'].rolling(window=14).min())) * 100

# On-Balance Volume
facDF['OBV'] = 0
comp_crsp['Change'] = comp_crsp.groupby('GVKEY')['PRC'].diff()
comp_crsp['Volume_Change'] = np.where(comp_crsp['Change'] > 0, comp_crsp['VOL'],  # Price increased
                                np.where(comp_crsp['Change'] < 0, -comp_crsp['VOL'], 0))  # Price decreased 
facDF['OBV'] = comp_crsp.groupby('GVKEY')['Volume_Change'].cumsum().shift().fillna(0)                                                 

# Gross Profit Over Asset
facDF['GPOA'] = (comp_crsp['REVTQ']-comp_crsp['COGSQ']) / comp_crsp['ATQ']

# ROA
facDF['ROA'] = comp_crsp['NIQ'] / comp_crsp['ATQ']

# Operating Profitability
facDF['OP'] = comp_crsp['OIADPQ'] / comp_crsp['REVTQ']

# Debt-Equity Ratio
facDF['DE'] = (comp_crsp['LCTQ']-comp_crsp['ACTQ']+comp_crsp['DLTTQ']) / comp_crsp['MktCap']

# Dupont Analysis
facDF['Dup'] = (comp_crsp['NIQ']/comp_crsp['REVTQ']) * (comp_crsp['REVTQ']/comp_crsp['ATQ']) * (comp_crsp['ATQ']/comp_crsp['SEQQ'])

# Liquidity Adjusted Debt
facDF['LIQ'] = (comp_crsp['LCTQ']-comp_crsp['ACTQ']+comp_crsp['DLTTQ']-comp_crsp['CHEQ']) / (comp_crsp['ATQ']-comp_crsp['INVTQ'])

# Asset to Market
facDF['AM'] = comp_crsp['ATQ'].shift(12) /  comp_crsp['MktCap'].shift(12)

# Debt to Market
facDF['DM']= (comp_crsp['DLCQ'].shift(12)+comp_crsp['DLTTQ'].shift(12)) / (comp_crsp['PRC']*comp_crsp['SHROUT']).shift(12)

# Tax Expense Surprise
facDF['TES'] = comp_crsp['TXTQ'] / (comp_crsp['CSHPRQ']*comp_crsp['AJEXQ']) - comp_crsp['TXTQ'].shift(12) / (comp_crsp['CSHPRQ'].shift(12)*comp_crsp['AJEXQ'].shift(12))
facDF['TES'] /= comp_crsp['ATQ'].shift(12) / (comp_crsp['CSHPRQ'].shift(12)*comp_crsp['AJEXQ'].shift(12))

# Net Stock Issues
facDF['NSI'] = np.log((comp_crsp['CSHOQ'].shift(12)*comp_crsp['AJEXQ'].shift(12)) / (comp_crsp['CSHOQ'].shift(24)*comp_crsp['AJEXQ'].shift(24)))

# Change in Net Operating Assets
facDF["NOA"] = (comp_crsp['ATQ']-comp_crsp['CHEQ']) - (comp_crsp['ATQ']-comp_crsp['DLCQ']-comp_crsp['DLTTQ']-comp_crsp['PSTKQ']-comp_crsp['CEQQ'])
facDF["DNOA"] = (facDF["NOA"]-facDF["NOA"].shift(12)) / comp_crsp['ATQ'].shift(12)

facDF = facDF.dropna()
facDF.reset_index(drop=True, inplace=True)
facDF

,GVKEY,indID,monthid,RET,ToQ,STO,OBV,GPOA,ROA,OP,DE,Dup,LIQ,AM,DM,TES,NSI,NOA,DNOA
0,001300,42,67,-0.081633,0.581505,10.526316,19934.0,0.067104,0.012637,0.073977,0.221399,0.038646,0.092302,1.821210,0.000429,-0.000259,-0.007651,5523.000,0.019892
1,001300,42,68,-0.068148,0.542712,0.000000,-19982.0,0.067104,0.012637,0.073977,0.241039,0.038646,0.092302,1.685045,0.000397,-0.000259,-0.007651,5523.000,0.019892
2,001300,42,69,-0.100806,0.483107,0.000000,-83420.0,0.067104,0.012637,0.073977,0.279078,0.038646,0.092302,1.792599,0.000423,-0.000259,-0.007651,5523.000,0.019892
3,001300,42,70,-0.098655,0.454066,0.000000,-120041.0,0.064823,0.011620,0.066897,0.360556,0.036369,0.117024,1.855183,0.000418,-0.001515,-0.000121,5782.000,0.048766
4,001300,42,71,0.122388,0.488934,22.105263,-174233.0,0.064823,0.011620,0.066897,0.326449,0.036369,0.117024,1.848786,0.000417,-0.001515,-0.000121,5782.000,0.048766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87571,011600,21,417,0.045821,0.839798,15.353694,766096.0,0.059727,0.015030,0.040625,0.065954,0.045399,0.018801,1.017785,0.000291,0.003291,-0.064536,1488.182,-0.022499
87572,011600,21,418,0.021082,0.871845,28.958803,827088.0,0.059727,0.015030,0.040625,0.063376,0.045399,0.018801,1.058080,0.000303,0.003291,-0.064536,1488.182,-0.022499
87573,011600,21,419,0.040478,0.905033,53.746756,871702.0,0.059727,0.015030,0.040625,0.060910,0.045399,0.018801,1.069831,0.000306,0.003291,-0.064536,1488.182,-0.022499
87574,011600,21,420,0.107572,1.051015,100.000000,900460.0,0.054002,-0.002005,0.016297,0.077594,-0.006061,0.068726,1.315359,0.000378,-0.005630,-0.062327,1481.488,-0.034958


### Uniformization

In [7]:
Fac = ["ToQ", "STO", "OBV", "GPOA", "ROA", "OP", "Dup", "AM", "DM", "TES", "NSI", "DNOA", "LIQ"]

for month, groupMonth in facDF.groupby(['monthid']):
    # for each month
    for ind, groupInd in groupMonth.groupby(['indID']):
        # for each industry
        for fac in Fac:
            # for each factor
            ind_avg = groupInd[fac].mean()
            ind_sd = groupInd[fac].std()
            for i in groupInd.index:
                if ind_sd!=0:
                    facDF.at[i, f'{fac}_z'] = (facDF.at[i, fac]-ind_avg) / ind_sd

facDF.reset_index(drop=True, inplace=True)
facDF['RET_t1'] = facDF['RET'].shift(-1)
facDF

,GVKEY,indID,monthid,RET,ToQ,STO,OBV,GPOA,ROA,OP,...,ROA_z,OP_z,Dup_z,AM_z,DM_z,TES_z,NSI_z,DNOA_z,LIQ_z,RET_t1
0,001300,42,67,-0.081633,0.581505,10.526316,19934.0,0.067104,0.012637,0.073977,...,-0.176404,0.121894,0.387704,0.050764,-0.136332,-0.396446,-0.276421,-0.147535,0.756999,-0.068148
1,001300,42,68,-0.068148,0.542712,0.000000,-19982.0,0.067104,0.012637,0.073977,...,-0.280412,0.049507,0.301592,1.324205,1.344439,-0.399838,0.381737,-0.326971,0.748164,-0.100806
2,001300,42,69,-0.100806,0.483107,0.000000,-83420.0,0.067104,0.012637,0.073977,...,-0.280412,0.049507,0.301592,1.517895,1.490557,-0.399838,0.381737,-0.326971,0.748164,-0.098655
3,001300,42,70,-0.098655,0.454066,0.000000,-120041.0,0.064823,0.011620,0.066897,...,-0.640115,-0.408130,-0.308778,1.321307,1.188311,-0.251571,0.641758,-0.508239,0.702979,0.122388
4,001300,42,71,0.122388,0.488934,22.105263,-174233.0,0.064823,0.011620,0.066897,...,-0.640115,-0.408130,-0.308778,1.372551,1.243244,-0.251571,0.643344,-0.508239,0.702979,-0.027027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87571,011600,21,417,0.045821,0.839798,15.353694,766096.0,0.059727,0.015030,0.040625,...,0.927670,-0.909648,1.463546,0.317560,0.334244,1.278639,-2.131184,-0.986739,-0.538390,0.021082
87572,011600,21,418,0.021082,0.871845,28.958803,827088.0,0.059727,0.015030,0.040625,...,0.832054,-0.274579,1.166353,-0.144870,0.020468,1.158598,-1.926422,-0.526719,-0.746729,0.040478
87573,011600,21,419,0.040478,0.905033,53.746756,871702.0,0.059727,0.015030,0.040625,...,0.832054,-0.274579,1.166353,-0.230548,-0.054692,1.158598,-1.926422,-0.526719,-0.746729,0.107572
87574,011600,21,420,0.107572,1.051015,100.000000,900460.0,0.054002,-0.002005,0.016297,...,-0.229656,-0.431998,-0.285407,-0.137773,-0.030054,-1.156076,-1.878359,-0.588379,-0.412741,-0.128023


### Prepare rolloing window for in-sample training

In [8]:
ret_matrix = sqldf(
    "SELECT i.GVKEY, i.monthid,\
    j.monthid as monthid2, j.RET, j.RET_t1, \
    j.ToQ_z as ToQ,j.STO_z as STO, j.OBV_z as OBV,\
    j.GPOA_z as GPOA, j.ROA_z as ROA, j.OP_z as OP,\
    j.Dup_z as Dup, j.AM_z as AM, j.DM_z as DM,\
    j.TES_z as TES, j.NSI_z as NSI,\
    j.DNOA_z as DNOA, j.LIQ_z as LIQ\
    FROM facDF AS i, facDF AS j\
    WHERE i.GVKEY = j.GVKEY AND i.monthid>j.monthid AND i.monthid <=j.monthid+300"
)

ret_matrix = ret_matrix.dropna()
ret_matrix = ret_matrix[ret_matrix['monthid'] > 300]
ret_matrix = ret_matrix.sort_values(['GVKEY', 'monthid', 'monthid2'])
ret_matrix.reset_index(drop=True, inplace=True)
ret_matrix

,GVKEY,monthid,monthid2,RET,RET_t1,ToQ,STO,OBV,GPOA,ROA,OP,Dup,AM,DM,TES,NSI,DNOA,LIQ
0,001004,324,24,-0.005319,0.197861,-0.377718,0.801641,-0.129626,-0.176479,-0.084466,0.073581,-0.169648,-0.343686,-0.317473,0.140427,0.036941,0.339473,-1.061405
1,001004,324,25,0.197861,0.085202,-0.274699,1.174341,-0.111333,-0.444124,0.259142,0.353814,0.125343,-0.437226,-0.292614,0.236682,0.079250,0.366893,-0.938741
2,001004,324,26,0.085202,0.107438,-0.246956,0.874460,-0.219716,-0.436106,0.272291,0.415965,0.126563,-0.523164,-0.315542,0.231342,0.078570,0.365769,-1.056016
3,001004,324,27,0.107438,-0.111940,-0.047655,0.954440,-0.276898,-0.283194,0.434132,0.429105,0.237317,-0.550178,-0.273413,0.261985,0.078731,0.319888,-0.939595
4,001004,324,28,-0.111940,0.025210,-0.312457,0.164140,-0.306798,-0.263435,0.511001,0.512555,0.322693,-0.431316,-0.225895,0.135802,0.127001,0.474991,-1.020123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7260062,160785,421,416,-0.065230,0.040539,-0.482652,0.277895,-0.913202,-1.014014,-0.236984,-1.243565,-0.261740,0.005967,0.013122,0.016839,-0.357895,0.413822,0.402328
7260063,160785,421,417,0.040539,-0.091288,-0.457375,0.775685,-0.881350,-1.014014,-0.236984,-1.243565,-0.261740,0.057028,0.047624,0.016839,-0.357895,0.413822,0.402328
7260064,160785,421,418,-0.091288,0.030810,-0.547778,0.131324,-0.904206,-0.480675,-0.328357,-0.237029,-0.559058,0.281984,0.174231,-1.214160,-0.445250,0.123485,0.209868
7260065,160785,421,419,0.030810,0.074722,-0.558803,0.124126,-0.905851,-0.498895,-0.365778,-0.258196,-0.601838,0.256026,0.203519,-1.199084,-0.445268,0.118258,0.223146


### Fama-MacBeth Cross-sectional Regression

Since factors were constructed accounting for lookahead bias (via lags when necessary), this step regresses factor values at time t to returns at time t

In [9]:
# Strage 1 - Calculate the beta of each factor
Fac = ["STO", "OBV", "AM", "DM", "Dup", "ToQ", "TES", "GPOA","DNOA", "NSI", "LIQ"]
betaDict = {
    'GVKEY':[], 'monthid':[], 'STO_beta':[], 'OBV_beta':[], 'AM_beta':[], 
    'DM_beta':[], 'Dup_beta':[], 'ToQ_beta':[], 'TES_beta':[], 'GPOA_beta':[], 
    "DNOA_beta":[], "NSI_beta":[], 'LIQ_beta':[]
}

for name, group in ret_matrix.groupby(['GVKEY', 'monthid']):
    stage1Test = linear_model.LinearRegression().fit(group[Fac], group["RET"])
    betaDict['GVKEY'].append(group['GVKEY'].iloc[0])
    betaDict['monthid'].append(group['monthid'].iloc[0])
    betaDict['STO_beta'].append(stage1Test.coef_[0])
    betaDict['OBV_beta'].append(stage1Test.coef_[1])
    betaDict['AM_beta'].append(stage1Test.coef_[2])
    betaDict['DM_beta'].append(stage1Test.coef_[3])
    betaDict['Dup_beta'].append(stage1Test.coef_[4])
    betaDict['ToQ_beta'].append(stage1Test.coef_[5])
    betaDict['TES_beta'].append(stage1Test.coef_[6])
    betaDict['GPOA_beta'].append(stage1Test.coef_[7])
    betaDict['DNOA_beta'].append(stage1Test.coef_[8])
    betaDict['NSI_beta'].append(stage1Test.coef_[9])
    betaDict['LIQ_beta'].append(stage1Test.coef_[10])

betaDF = pd.DataFrame(betaDict)
betaDF = pd.merge(betaDF, facDF[['GVKEY', 'monthid', 'RET', 'RET_t1']], on=['GVKEY', 'monthid'], how='inner')
betaDF = betaDF.dropna()
betaDF.reset_index(drop=True, inplace=True)

# Stage 2 - Calculate lambda of each factor
lambdaDict = {
    'GVKEY':[], 'monthid':[], 'STO_lambda':[], 'OBV_lambda':[], 'AM_lambda':[], 
    'DM_lambda':[], 'Dup_lambda':[], 'ToQ_lambda':[], 'TES_lambda':[],
    'GPOA_lambda':[], 'DNOA_lambda':[], 'NSI_lambda':[], 'LIQ_lambda':[]}

for name, group in betaDF.groupby(['monthid']):
    stage2Test = linear_model.LinearRegression().fit(
        group[["STO_beta", "OBV_beta", "AM_beta", "DM_beta", "Dup_beta", "ToQ_beta", "TES_beta", "GPOA_beta", "DNOA_beta", "NSI_beta", "LIQ_beta"]], 
        group["RET"]
    )
    lambdaDict['GVKEY'].append(group['GVKEY'].iloc[0])
    lambdaDict['monthid'].append(group['monthid'].iloc[0])
    lambdaDict['STO_lambda'].append(stage2Test.coef_[0])
    lambdaDict['OBV_lambda'].append(stage2Test.coef_[1])
    lambdaDict['AM_lambda'].append(stage2Test.coef_[2])
    lambdaDict['DM_lambda'].append(stage2Test.coef_[3])
    lambdaDict['Dup_lambda'].append(stage2Test.coef_[4])
    lambdaDict['ToQ_lambda'].append(stage2Test.coef_[5])
    lambdaDict['TES_lambda'].append(stage2Test.coef_[6])
    lambdaDict['GPOA_lambda'].append(stage2Test.coef_[7])
    lambdaDict['DNOA_lambda'].append(stage2Test.coef_[8])
    lambdaDict['NSI_lambda'].append(stage2Test.coef_[9])
    lambdaDict['LIQ_lambda'].append(stage2Test.coef_[10])

lambdaDF = pd.DataFrame(lambdaDict)

In [10]:
# Calculate t-statistics of each factor
def MacBethStat(col):
    avg = col.mean()
    std = col.std()
    n = len(col)
    tstat = avg/(std/np.sqrt(n))

    return avg, tstat

FM_facTStat = {'Factor':[], 'AVG':[], 'tStat':[], 'abs(tStat)':[]}
for fac in Fac:
    FM_facTStat['Factor'].append(fac)
    avg, t = MacBethStat(lambdaDF[f'{fac}_lambda'])
    FM_facTStat['AVG'].append(avg)
    FM_facTStat['tStat'].append(t)
    FM_facTStat['abs(tStat)'].append(np.abs(t))

FM_facTStat = pd.DataFrame(FM_facTStat)
FM_facTStat = FM_facTStat.sort_values(['abs(tStat)'], ascending=False)
FM_facTStat.reset_index(drop=True, inplace=True)
FM_facTStat

,Factor,AVG,tStat,abs(tStat)
0,TES,-0.082234,-1.722906,1.722906
1,ToQ,0.020846,1.233038,1.233038
2,Dup,0.037937,1.105471,1.105471
3,DNOA,0.052101,0.928276,0.928276
4,STO,0.075796,0.765142,0.765142
5,LIQ,-0.007550,-0.707060,0.707060
6,OBV,0.005818,0.571455,0.571455
7,NSI,-0.015710,-0.356673,0.356673
8,AM,-0.004808,-0.308986,0.308986
9,GPOA,0.002446,0.145981,0.145981


### Random Forest Model

In [11]:
from sklearn.ensemble import RandomForestRegressor

Fac = ["STO", "OBV", "AM", "DM", "Dup", "ToQ", "TES", "GPOA", "DNOA", "NSI", "LIQ"]
rf = RandomForestRegressor(max_depth=15, max_features=7, n_estimators=1, n_jobs=-1)
feaImpDict = {'GVKEY':[], 'monthid':[], 'STO':[], 'OBV':[], 'AM':[], 'DM':[], 'Dup':[], 'ToQ':[], 'TES':[], 'GPOA':[], 'DNOA':[], 'NSI':[], 'LIQ':[]}

for name, group in ret_matrix.groupby(['GVKEY', 'monthid']):
    rf.fit(group[Fac], group['RET'])
    feaImpDict['GVKEY'].append(group['GVKEY'].iloc[0])
    feaImpDict['monthid'].append(group['monthid'].iloc[0])
    feaImpDict['STO'].append(rf.feature_importances_[0])
    feaImpDict['OBV'].append(rf.feature_importances_[1])
    feaImpDict['AM'].append(rf.feature_importances_[2])
    feaImpDict['DM'].append(rf.feature_importances_[3])
    feaImpDict['Dup'].append(rf.feature_importances_[4])
    feaImpDict['ToQ'].append(rf.feature_importances_[5])
    feaImpDict['TES'].append(rf.feature_importances_[6])
    feaImpDict['GPOA'].append(rf.feature_importances_[7])
    feaImpDict['DNOA'].append(rf.feature_importances_[8])
    feaImpDict['NSI'].append(rf.feature_importances_[9])
    feaImpDict['LIQ'].append(rf.feature_importances_[10])

feaImpDF = pd.DataFrame(feaImpDict)
RF_facTStat = {'Factor':[], 'AVG':[]}
for fac in Fac:
    RF_facTStat['Factor'].append(fac)
    RF_facTStat['AVG'].append(feaImpDF[fac].mean())

RF_facTStat = pd.DataFrame(RF_facTStat)
RF_facTStat = RF_facTStat.sort_values(['AVG'], ascending=[False])
RF_facTStat.reset_index(drop=True, inplace=True)
RF_facTStat

,Factor,AVG
0,STO,0.204004
1,OBV,0.110348
2,ToQ,0.098615
3,AM,0.081693
4,DM,0.079778
5,DNOA,0.074926
6,NSI,0.073464
7,TES,0.072087
8,Dup,0.071676
9,LIQ,0.066835


## Out-of-sample Testing

### Fama-MacBeth Linear Regression

In [12]:
# Predict returns
selectedFac = ["OBV", "AM", "LIQ", "TES", "STO"]
test_betaDict = {'GVKEY':[], 'monthid':[], 'alpha':[], 'OBV_beta':[], 'AM_beta':[],'LIQ_beta':[], 'TES_beta':[], 'STO_beta':[]}

for name, group in ret_matrix.groupby(['GVKEY', 'monthid']):
    stage1Model = linear_model.LinearRegression().fit(group[selectedFac], group["RET"])
    test_betaDict['GVKEY'].append(group['GVKEY'].iloc[0])
    test_betaDict['monthid'].append(group['monthid'].iloc[0])
    test_betaDict['alpha'].append(stage1Model.intercept_)
    test_betaDict['OBV_beta'].append(stage1Model.coef_[0])
    test_betaDict['AM_beta'].append(stage1Model.coef_[1])
    test_betaDict['LIQ_beta'].append(stage1Model.coef_[2])
    test_betaDict['TES_beta'].append(stage1Model.coef_[3])
    test_betaDict['STO_beta'].append(stage1Model.coef_[4])

test_betaDF = pd.DataFrame(test_betaDict)
FM_testingDF = pd.merge(test_betaDF, facDF[['GVKEY', 'monthid', 'RET', 'OBV_z', 'AM_z', 'LIQ_z', 'TES_z', 'STO_z']], on=['GVKEY', 'monthid'], how='inner')
FM_testingDF['predRet'] = FM_testingDF['alpha']
for fac in selectedFac:
    FM_testingDF['predRet'] += FM_testingDF[f'{fac}_beta'] * FM_testingDF[f'{fac}_z']

# Construct hedge portfolio
FM_testingDF['rank'] = FM_testingDF.groupby(['monthid'])['predRet'].transform(lambda x:pd.qcut(x, 10, labels=False)) + 1

FM_top10 = {'monthid':[], 'RET':[], 'predRet':[]}
for _, group in FM_testingDF[FM_testingDF['rank']==10].groupby(['monthid']):
    FM_top10['monthid'].append(group['monthid'].iloc[0])
    FM_top10['RET'].append(group['RET'].mean())
    FM_top10['predRet'].append(group['predRet'].mean())
FM_top10DF = pd.DataFrame(FM_top10)

FM_bottom10 = {'monthid':[], 'RET':[], 'predRet':[]}
for _, group in FM_testingDF[FM_testingDF['rank']==1].groupby(['monthid']):
    FM_bottom10['monthid'].append(group['monthid'].iloc[0])
    FM_bottom10['RET'].append(group['RET'].mean())
    FM_bottom10['predRet'].append(group['predRet'].mean())
FM_bottom10DF = pd.DataFrame(FM_bottom10)

FM_hedgeDF = pd.merge(FM_top10DF, FM_bottom10DF, on=['monthid'])
FM_hedgeDF['RET'] = FM_hedgeDF['RET_x'] - FM_hedgeDF['RET_y']
FM_hedgeDF['predRet'] = FM_hedgeDF['predRet_x'] - FM_hedgeDF['predRet_y']
FM_hedgeDF = FM_hedgeDF.drop(['RET_x', 'RET_y', 'predRet_x', 'predRet_y'], axis=1)
FM_hedgeDF.to_excel('FM_hedgeDF.xlsx')
FM_hedgeDF


,monthid,RET,predRet
0,301,0.057758,0.102473
1,302,0.051589,0.103314
2,303,0.036105,0.095355
3,304,0.096710,0.106434
4,305,0.061435,0.121859
...,...,...,...
116,417,0.035703,0.113445
117,418,0.077796,0.136552
118,419,0.040018,0.115516
119,420,0.026580,0.114803


### Random Forest

In [13]:
# Predict returns
rf = RandomForestRegressor(max_depth=15, max_features=7, n_estimators=1, n_jobs=-1)
rfFac = ['OBV_z', 'AM_z', 'LIQ_z', 'TES_z', 'STO_z']
rfDF = facDF[['GVKEY', 'monthid', 'RET', 'OBV_z', 'AM_z', 'LIQ_z', 'TES_z', 'STO_z']].copy().dropna()
RF_testing = {'GVKEY':[], 'monthid':[], 'RET':[], 'predRet':[]}

for name, group in rfDF.groupby(['GVKEY']):
    for i in range(301, 421):
        test = group[group['monthid']==i]
        if test.empty: continue
        test_x = test[rfFac]

        train = group[(group['monthid'] >= i-300) & (group['monthid']<i)]
        if train.empty: continue
        train_x = train[rfFac]
        train_y = train['RET']
        
        rf.fit(train_x, train_y)
        RF_testing['GVKEY'].append(test['GVKEY'].iloc[0])
        RF_testing['monthid'].append(test['monthid'].iloc[0])
        RF_testing['RET'].append(test['RET'].iloc[0])
        RF_testing['predRet'].append(rf.predict(test_x)[0])

RF_testingDF = pd.DataFrame(RF_testing)

# Construct hedge portfolio
RF_testingDF['rank'] = RF_testingDF.groupby(['monthid'])['predRet'].transform(lambda x:pd.qcut(x, 10, labels=False)) + 1

RF_top10 = {'monthid':[], 'RET':[], 'predRet':[]}
for _, group in RF_testingDF[RF_testingDF['rank']==10].groupby(['monthid']):
    RF_top10['monthid'].append(group['monthid'].iloc[0])
    RF_top10['RET'].append(group['RET'].mean())
    RF_top10['predRet'].append(group['predRet'].mean())
RF_top10DF = pd.DataFrame(RF_top10)

RF_bottom10 = {'monthid':[], 'RET':[], 'predRet':[]}
for _, group in RF_testingDF[RF_testingDF['rank']==1].groupby(['monthid']):
    RF_bottom10['monthid'].append(group['monthid'].iloc[0])
    RF_bottom10['RET'].append(group['RET'].mean())
    RF_bottom10['predRet'].append(group['predRet'].mean())
RF_bottom10DF = pd.DataFrame(RF_bottom10)

RF_hedgeDF = pd.merge(RF_top10DF, RF_bottom10DF, on=['monthid'])
RF_hedgeDF['RET'] = RF_hedgeDF['RET_x'] - RF_hedgeDF['RET_y']
RF_hedgeDF['predRet'] = RF_hedgeDF['predRet_x'] - RF_hedgeDF['predRet_y']
RF_hedgeDF = RF_hedgeDF.drop(['RET_x', 'RET_y', 'predRet_x', 'predRet_y'], axis=1)
RF_hedgeDF.to_excel('RF_hedgeDF.xlsx')
RF_hedgeDF

,monthid,RET,predRet
0,301,-0.004882,0.328815
1,302,0.051626,0.424062
2,303,0.070700,0.318033
3,304,0.037665,0.374994
4,305,0.051014,0.375305
...,...,...,...
115,416,0.011420,0.377815
116,417,0.011256,0.366055
117,418,0.043422,0.374634
118,419,-0.013847,0.293477


### Combined

In [14]:
combinedDF = pd.merge(FM_testingDF[['GVKEY', 'monthid', 'RET', 'predRet']], RF_testingDF[['GVKEY', 'monthid', 'RET', 'predRet']], on=['GVKEY', 'monthid'], how = 'inner')
combinedDF['RET'] = (combinedDF['RET_x']+combinedDF['RET_y']) / 2
combinedDF['predRet'] = (combinedDF['predRet_x']+combinedDF['predRet_y']) / 2
combinedDF = combinedDF.drop(['RET_x', 'RET_y', 'predRet_x', 'predRet_y'], axis=1)

# ranking
combinedDF['rank'] = combinedDF.groupby(['monthid'])['predRet'].transform(lambda x:pd.qcut(x, 10, labels=False)) + 1

combined_top10 = {'monthid':[], 'RET':[], 'predRet':[]}
for _, group in combinedDF[combinedDF['rank']==10].groupby(['monthid']):
    combined_top10['monthid'].append(group['monthid'].iloc[0])
    combined_top10['RET'].append(group['RET'].mean())
    combined_top10['predRet'].append(group['predRet'].mean())
combined_top10DF = pd.DataFrame(combined_top10)

combined_bottom10 = {'monthid':[], 'RET':[], 'predRet':[]}
for _, group in combinedDF[combinedDF['rank']==1].groupby(['monthid']):
    combined_bottom10['monthid'].append(group['monthid'].iloc[0])
    combined_bottom10['RET'].append(group['RET'].mean())
    combined_bottom10['predRet'].append(group['predRet'].mean())
combined_bottom10DF = pd.DataFrame(combined_bottom10)

combined_hedgeDF = pd.merge(combined_top10DF, combined_bottom10DF, on=['monthid'])
combined_hedgeDF['RET'] = combined_hedgeDF['RET_x'] - combined_hedgeDF['RET_y']
combined_hedgeDF['predRet'] = combined_hedgeDF['predRet_x'] - combined_hedgeDF['predRet_y']
combined_hedgeDF = combined_hedgeDF.drop(['RET_x', 'RET_y', 'predRet_x', 'predRet_y'], axis=1)
combined_hedgeDF.to_excel('combined_hedgeDF.xlsx')
combined_hedgeDF

,monthid,RET,predRet
0,301,0.010416,0.178221
1,302,0.052175,0.234513
2,303,0.055561,0.164860
3,304,0.050231,0.202460
4,305,0.054585,0.209001
...,...,...,...
115,416,0.035024,0.221746
116,417,0.021452,0.215814
117,418,0.059432,0.224356
118,419,0.018325,0.168939


## Analyze Performence

In [15]:
ff = pd.read_sas("ff.sas7bdat", encoding = "ISO-8859-1")
ff['monthid'] = (ff.DATEFF.dt.year-1985)*12 + ff.DATEFF.dt.month
ff = ff.drop(['DATEFF'], axis=1)

### Fama-MacBeth Linear Regression

In [16]:
FM_perfStat = {'Raw Return':[], 'Sharpe Ratio':[], 'CAPM alpha':[], 'FF4 alpha':[], 'Information Ratio':[]}

FM_portDF = pd.merge(FM_hedgeDF, ff, on=['monthid'], how='inner')
FM_portDF['XRET'] = FM_portDF['RET'] - FM_portDF['RF']
FM_portDF['XMKT'] = FM_portDF['RET'] - (FM_portDF["MKTRF"]+FM_portDF['RF'])

FM_capm = sm.add_constant(FM_portDF["MKTRF"])
FM_CAPMmodel = sm.OLS(FM_portDF['XRET'], FM_capm).fit()
FM_ff4 = sm.add_constant(FM_portDF[["MKTRF", "SMB", "HML", "UMD"]])
FM_ff4model = sm.OLS(FM_portDF['XRET'], FM_ff4).fit()

FM_perfStat['Raw Return'].append(FM_portDF['RET'].sum())
FM_perfStat['Sharpe Ratio'].append(12*(FM_portDF['XRET'].mean()) / (np.sqrt(12)*FM_portDF['XRET'].std()))
FM_perfStat['CAPM alpha'].append(FM_CAPMmodel.params[0])
FM_perfStat['FF4 alpha'].append(FM_ff4model.params[0])
FM_perfStat['Information Ratio'].append(12*(FM_portDF['XMKT'].mean()) / (np.sqrt(12)*FM_portDF['XMKT'].std()))
FM_perfStatDF = pd.DataFrame(FM_perfStat)
FM_perfStatDF

,Raw Return,Sharpe Ratio,CAPM alpha,FF4 alpha,Information Ratio
0,7.692509,6.804745,0.065098,0.063192,3.349305


### Random Forest

In [17]:
RF_perfStat = {'Raw Return':[], 'Sharpe Ratio':[], 'CAPM alpha':[], 'FF4 alpha':[], 'Information Ratio':[]}

RF_portDF = pd.merge(RF_hedgeDF, ff, on=['monthid'], how='inner')
RF_portDF['XRET'] = RF_portDF['RET'] - RF_portDF['RF']
RF_portDF['XMKT'] = RF_portDF['RET'] - (RF_portDF["MKTRF"]+RF_portDF['RF'])

RF_capm = sm.add_constant(RF_portDF["MKTRF"])
RF_CAPMmodel = sm.OLS(RF_portDF['XRET'], RF_capm).fit()
RF_ff4 = sm.add_constant(RF_portDF[["MKTRF", "SMB", "HML", "UMD"]])
RF_ff4model = sm.OLS(RF_portDF['XRET'], RF_ff4).fit()

RF_perfStat['Raw Return'].append(RF_portDF['RET'].sum())
RF_perfStat['Sharpe Ratio'].append(12*(RF_portDF['XRET'].mean()) / (np.sqrt(12)*RF_portDF['XRET'].std()))
RF_perfStat['CAPM alpha'].append(RF_CAPMmodel.params[0])
RF_perfStat['FF4 alpha'].append(RF_ff4model.params[0])
RF_perfStat['Information Ratio'].append(12*(RF_portDF['XMKT'].mean()) / (np.sqrt(12)*RF_portDF['XMKT'].std()))
RF_perfStatDF = pd.DataFrame(RF_perfStat)
RF_perfStatDF

,Raw Return,Sharpe Ratio,CAPM alpha,FF4 alpha,Information Ratio
0,3.37883,3.377166,0.029537,0.029388,1.126204


### Combine portfolio

In [18]:
combined_perfStat = {'Raw Return':[], 'Sharpe Ratio':[], 'CAPM alpha':[], 'FF4 alpha':[], 'Information Ratio':[]}

combined_portDF = pd.merge(combined_hedgeDF, ff, on=['monthid'], how='inner')
combined_portDF['XRET'] = combined_portDF['RET'] - combined_portDF['RF']
combined_portDF['XMKT'] = combined_portDF['RET'] - (combined_portDF["MKTRF"] + combined_portDF['RF'])

combined_capm = sm.add_constant(combined_portDF["MKTRF"])
combined_CAPMmodel = sm.OLS(combined_portDF['XRET'], combined_capm).fit()
combined_ff4 = sm.add_constant(combined_portDF[["MKTRF", "SMB", "HML", "UMD"]])
combined_ff4model = sm.OLS(combined_portDF['XRET'], combined_ff4).fit()

combined_perfStat['Raw Return'].append(combined_portDF['RET'].sum())
combined_perfStat['Sharpe Ratio'].append(12*(combined_portDF['XRET'].mean()) / (np.sqrt(12)*combined_portDF['XRET'].std()))
combined_perfStat['CAPM alpha'].append(combined_CAPMmodel.params[0])
combined_perfStat['FF4 alpha'].append(combined_ff4model.params[0])
combined_perfStat['Information Ratio'].append(12*(combined_portDF['XMKT'].mean()) / (np.sqrt(12)*combined_portDF['XMKT'].std()))
combined_perfStatDF = pd.DataFrame(combined_perfStat)
combined_perfStatDF

,Raw Return,Sharpe Ratio,CAPM alpha,FF4 alpha,Information Ratio
0,5.298281,5.164136,0.045688,0.045054,2.16088
